# Master's Thesis


# Abstract

**Objectives:** This study will extend an established model 1 ​for estimating the current living
wage in 2015 to the past decade for the purpose of:

* an exploratory analysis trends in the gap between the estimated living wage and the
minimum wage
* evaluating any correlation between the living wage gap and other economic metrics,
including public funds spent on social services

**Methods:** The original data set for this model is for 2015. This study will extend the data
sources of this model into the past to enable trend analysis. Data for economic metrics from
public data sources will supplement this data for correlation analysis.

# Methods

## Model

The original model proposed estimated the living wage in terms of 9 variables:

Basic needs budget = Food _cost + child _care _cost + (insurance premiums + health care costs) + housing _cost + transportation _cost + other_necessities_cost

Living wage = Basic needs budget + ( basic needs budget * tax rate )

## Data Sources

The following data sources are used to find estimates of the model variables:

* The food cost is estimated from data from the USDA’s low-cost food plan national
average in June 2014. 2
* Child care is based off state-level estimates published by the National Association of
Child Care Resource and Referral Agencies. 3
* Insurance costs are based on the insurance component of the 2013 Medical
Expenditure Panel Survey. 4
* Housing costs are estimated from the HUD Fair Market Rents (FMR) estimates
* Other variables are pulled from the 2014 Bureau of Labor Statistics Consumer
Expenditure Survey. 5

These data sets extend into the past, allowing for calculating the model for years past. The data will also have to be adjusted for inflation 6.

## Analytic Approach

First, data will be gathered from the data sources of the original model but will be extended into the past. The methodology followed by the model will be replicated to come up with a data set representing estimates of the living wage across time. After the data set is prepared, the trend of the living wage as compared to minimum wage can be examined. Has the gap increased or decreased over time, and at what rate? Have certain areas seen larger than average increases or decreases in this gap? Once preliminary trend analysis is done, this data set will be analyzed in comparison to other economic trends to see if any interesting correlations can be found, including public expenditure for social services, economic growth, unemployment, etc.

## Presentation Of Results

Results will be presented for both parts of the data analysis. TODO: expand

## Background / Sources

- Glasmeier AK, Nadeau CA, Schultheis E: LIVING WAGE CALCULATOR User’s Guide / Technical Notes 2014 Update
- USDA low-cost food plan, June, 2014
- Child Care in America 2014 State fact sheets
- 2013 Medical Expenditure Panel Survey Available
- Consumer Expenditure Survey
- Inflation Calculator